In [ ]:
pip install pandas sumy transformers torch datasets


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install nltk scikit-learn
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download the 'punkt_tab' resource

import os
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
import numpy as np

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extractive_summary_tfidf(text, num_sentences=3):
    """
    Extractive summarization using TF-IDF and cosine similarity.
    """
    # Sentence tokenization
    sentences = sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return text  # Not enough sentences to summarize

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Cosine similarity matrix
    sim_matrix = cosine_similarity(tfidf_matrix)

    # Sentence scores: sum of similarities
    scores = sim_matrix.sum(axis=1)

    # Top N sentence indices
    ranked_indices = np.argsort(scores)[-num_sentences:]
    ranked_indices.sort()  # To maintain original order in text

    # Build summary
    summary = ' '.join([sentences[i] for i in ranked_indices])
    return summary

# Load dataset
df = pd.read_csv('/content/dataset_nlp new.csv', encoding='latin-1')

# Apply Extractive Summarization
df["extractive_summary"] = df["abstract"].apply(lambda x: extractive_summary_tfidf(str(x)))

# Save Extractive Summaries
os.makedirs("/mnt/data", exist_ok=True)
df.to_csv("/mnt/data/extractive_summaries.csv", index=False)

print("✅ Extractive summaries (TF-IDF + Cosine Similarity) saved as extractive_summaries.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


✅ Extractive summaries (TF-IDF + Cosine Similarity) saved as extractive_summaries.csv


In [ ]:
from google.colab import files
files.download("/mnt/data/extractive_summaries.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os
import pandas as pd

# Load T5 Model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


def abstractive_summary_batch(texts, max_length=100, batch_size=8):
    summaries = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        inputs = tokenizer(["summarize: " + text for text in batch], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        summary_ids = model.generate(inputs.input_ids, max_length=max_length, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
        summaries.extend(tokenizer.batch_decode(summary_ids, skip_special_tokens=True))
    return summaries

df = pd.read_csv('/content/dataset_nlp new.csv', encoding='latin-1')
# Apply Abstractive Summarization in Batches
df["abstractive_summary"] = abstractive_summary_batch(df["abstract"].astype(str).tolist())
os.makedirs("/mnt/data", exist_ok=True)

# Save Faster Abstractive Summaries
df.to_csv("/mnt/data/abstractive_summaries.csv", index=False)
print("✅ Faster abstractive summaries saved as abstractive_summaries.csv")
print(os.path.abspath("/mnt/data/abstractive_summaries.csv"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Faster abstractive summaries saved as abstractive_summaries.csv
/mnt/data/abstractive_summaries.csv


In [ ]:
from google.colab import files
files.download("/mnt/data/abstractive_summaries.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Load and clean dataset
df = pd.read_csv('/content/dataset_nlp new.csv', encoding='latin-1')
df = df.dropna(subset=["abstract", "title"])
df["abstract"] = df["abstract"].fillna("")
df["title"] = df["title"].fillna("")

# Rename and convert to dataset
df = df[["abstract", "title"]].rename(columns={"abstract": "text", "title": "summary"})
dataset = Dataset.from_pandas(df.reset_index(drop=True))

# Preprocessing function
def preprocess_function(examples):
    inputs = ["summarize: " + str(text) for text in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True, padding="max_length")

    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Training args (no evaluation_strategy)
training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10_000,
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train and evaluate manually
trainer.train()
trainer.evaluate()

print("✅ Model fine-tuned successfully!")


Map:   0%|          | 0/2122 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dicapriyogandhi (dicapriyogandhi-indian-institute-of-technology-patna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,3.118800
1000,2.753900


✅ Model fine-tuned successfully!


In [ ]:
import os

model_path_fine_tuned = "./t5_finetuned"
if os.path.exists(model_path_fine_tuned):
    print("✅ Fine-tuned model directory exists.")
    print("📂 Files inside:", os.listdir(model_path_fine_tuned))
else:
    print("❌ Model directory not found. Fine-tuning might have failed.")


✅ Fine-tuned model directory exists.
📂 Files inside: ['checkpoint-1275']


In [ ]:
pip install rouge-score pandas torch transformers


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1dd99388677bc87f795874032086c0e727899e0af038a572a37557fb6019ecc5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the summarization datasets
extractive_df = pd.read_csv("/mnt/data/extractive_summaries.csv")
abstractive_df = pd.read_csv("/mnt/data/abstractive_summaries.csv")

# Define ROUGE Scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def evaluate_summaries(df, summary_col, reference_col="abstract"):
    """
    Computes ROUGE Precision, Recall, and F1-score for the summaries.
    """
    results = {"rouge1": [], "rouge2": [], "rougeL": []}

    for ref, summary in zip(df[reference_col], df[summary_col]):
        if pd.isna(ref) or pd.isna(summary):  # Skip missing values
            continue
        scores = scorer.score(ref, summary)

        for rouge_type in results.keys():
            results[rouge_type].append({
                "precision": scores[rouge_type].precision,
                "recall": scores[rouge_type].recall,
                "f1": scores[rouge_type].fmeasure
            })

    # Compute average scores
    avg_scores = {}
    for rouge_type in results.keys():
        avg_scores[rouge_type] = {
            "precision": sum(d["precision"] for d in results[rouge_type]) / len(results[rouge_type]),
            "recall": sum(d["recall"] for d in results[rouge_type]) / len(results[rouge_type]),
            "f1": sum(d["f1"] for d in results[rouge_type]) / len(results[rouge_type])
        }

    return avg_scores

# Evaluate Extractive Summarization
extractive_scores = evaluate_summaries(extractive_df, "extractive_summary")

# Evaluate Abstractive Summarization
abstractive_scores = evaluate_summaries(abstractive_df, "abstractive_summary")

# Print Scores
print("\n🔹 **Extractive Summarization Scores:**")
print(pd.DataFrame(extractive_scores))

print("\n🔹 **Abstractive Summarization Scores:**")
print(pd.DataFrame(abstractive_scores))



🔹 **Extractive Summarization Scores:**
             rouge1    rouge2    rougeL
precision  1.000000  0.989174  1.000000
recall     0.595819  0.588146  0.595819
f1         0.726092  0.717157  0.726092

🔹 **Abstractive Summarization Scores:**
             rouge1    rouge2    rougeL
precision  0.985047  0.936118  0.965976
recall     0.364035  0.341841  0.357057
f1         0.514045  0.483952  0.504249
